# Single Trial Parametrization
<p>created: May 20, 2024 <br>
last modified: May 29, 2024 </p>
<p>
Compute Spectral Parametrization for single trials with and without knee (aperiodic mode)


In [7]:
# general
import os
import numpy as np
import pandas as pd

# lab imports
from specparam import SpectralGroupModel
from specparam.objs import fit_models_3d, combine_model_objs
from specparam.sim import sim_group_power_spectra
from specparam.sim.utils import create_freqs
from specparam.sim.params import param_sampler
from specparam.utils.io import load_group

# custom
import sys
sys.path.append("../../code")
from settings import SPECPARAM_SETTINGS, N_JOBS
from info import SESSIONS, N_ARRAYS, N_CHANS

### load example file

In [5]:
# load example data
data = np.load('E:\data\lfp\lfp_psd\A_SNR_041018_spectra.npz')
print(data.files)

['spectra_pre', 'spectra_post', 'freq']


In [9]:
# set-up as [n_conditions, n_channels, n_freqs]
print(f"spectra_pre shape: {data['spectra_pre'].shape} \nspectra_post shape: {data['spectra_post'].shape}")

spectra_pre shape: (102, 1024, 76) 
spectra_post shape: (102, 1024, 76)


In [11]:
# initialize model and fit the 3D array of power spectra (pre, with knee)
fg = SpectralGroupModel(**SPECPARAM_SETTINGS)
fgs = fit_models_3d(fg, data['freq'], data['spectra_pre'])

c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.py:67: RuntimeWarning: invalid value encountered in log10
  ys = ys + offset - np.log10(knee + xs**exp)
c:\Users\Andre\anaconda3\envs\voytek_lab\lib\site-packages\specparam\core\funcs.

In [ ]:
# [test] initialize model and fit the 3D array of power spectra (pre, without knee)
fg = SpectralGroupModel(**SPECPARAM_SETTINGS, aperiodic_mode='fixed')
fgs = fit_models_3d(fg, data['freq'], data['spectra_pre'], n_jobs=N_JOBS)

array([[[4.51328789e+03, 1.73108534e+04, 1.77866364e+04, ...,
         2.57020746e+00, 2.71163605e+00, 1.31326432e+00],
        [2.56981409e+04, 5.94112063e+04, 6.11241257e+04, ...,
         4.74229225e+00, 4.79358053e+00, 2.40821861e+00],
        [5.14884721e+03, 1.58649014e+04, 1.71773547e+04, ...,
         6.97628992e-01, 7.14045918e-01, 3.50167652e-01],
        ...,
        [2.05484083e+04, 3.64936181e+04, 3.66940021e+04, ...,
         1.38782546e+00, 1.35591897e+00, 6.88753147e-01],
        [2.53030665e+04, 4.90492602e+04, 4.60879535e+04, ...,
         2.18204625e+00, 2.15380428e+00, 1.01662215e+00],
        [2.31681283e+04, 4.51709291e+04, 3.91878438e+04, ...,
         3.78948975e+00, 3.80165945e+00, 1.92812513e+00]],

       [[1.46254191e+04, 3.32931070e+04, 3.74000960e+04, ...,
         2.94492376e+01, 2.96218170e+01, 1.51065492e+01],
        [2.28609196e+04, 4.64371415e+04, 4.24423309e+04, ...,
         3.20434201e+00, 3.18870314e+00, 1.61722508e+00],
        [9.87546071e+03, 

In [8]:
df_data = pd.DataFrame({
                'session'   :   np.repeat('session', N_ARRAYS*N_CHANS),
                'channel'   :   np.concatenate([np.arange(N_CHANS)] * N_ARRAYS),
                'chan_idx'  :   np.arange(N_ARRAYS*N_CHANS),
                'array'     :   np.repeat(np.arange(N_ARRAYS), N_CHANS) + 1,
                'epoch'     :   np.repeat('epoch', N_ARRAYS*N_CHANS)})
            

In [12]:
modes = ['fixed', 'knee']
ap_mode = np.array(np.repeat(modes, df_data.shape[0]))
ap_mode

array(['fixed', 'fixed', 'fixed', ..., 'knee', 'knee', 'knee'],
      dtype='<U5')